In [ ]:
import pandas as pd
import requests
import time
from geopy.distance import geodesic

# Load the CSV file containing standardized ZIP codes (CEPs)
df = pd.read_csv("validated_ceps_for_geocoding.csv")

# Create empty columns for latitude and longitude
df['latitude'] = None
df['longitude'] = None

# Function to get latitude and longitude from Nominatim using a query (CEP or address)
def get_lat_lon_nominatim(query):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': query,
        'countrycodes': 'br',
        'format': 'json',
        'addressdetails': 1,
        'limit': 1
    }
    headers = {
        'User-Agent': 'CEP-Geocoder/1.0 (youremail@example.com)'  # customize with your email
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        if response.status_code == 200 and response.json():
            data = response.json()[0]
            return float(data['lat']), float(data['lon'])
    except Exception as e:
        print(f"Error accessing Nominatim: {e}")
    return None, None

# Fallback function using ViaCEP to construct an address and geocode it
def fallback_via_cep(cep):
    cep = str(cep).zfill(8)
    url = f'https://viacep.com.br/ws/{cep}/json/'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200 and 'erro' not in response.text:
            data = response.json()
            street = data.get('logradouro', '')
            city = data.get('localidade', '')
            state = data.get('uf', '')
            address = f"{street}, {city}, {state}"
            print(f" -> Address via ViaCEP (without neighborhood): {address}")
            lat, lon = get_lat_lon_nominatim(address)
            print(f" -> Fallback result: Latitude={lat}, Longitude={lon}")
            return lat, lon
        else:
            print(f" -> ViaCEP did not find the CEP {cep}")
    except Exception as e:
        print(f" -> Error accessing ViaCEP for {cep}: {e}")
    return None, None

# Main loop to geocode all ZIP codes
for idx, row in df.iterrows():
    cep = str(row['CEP']).zfill(8)
    lat, lon = get_lat_lon_nominatim(cep)

    if lat is None or lon is None:
        print(f"{idx+1}/{len(df)} - CEP {cep} not found in Nominatim. Trying ViaCEP fallback...")
        lat, lon = fallback_via_cep(cep)

    df.at[idx, 'latitude'] = lat
    df.at[idx, 'longitude'] = lon
    print(f"{idx+1}/{len(df)} - CEP: {cep} -> Latitude: {lat}, Longitude: {lon}")
    time.sleep(1)  # Respect API rate limits

# Save CSV with geocoded coordinates
df.to_csv("employees_with_latlong_raw.csv", index=False)
print("✅ File saved as 'employees_with_latlong_raw.csv'")

# Load coordinates of workplace based on its ZIP code
workplace_cep = "05425070"
lat_workplace, lon_workplace = get_lat_lon_nominatim(workplace_cep)
print(f"Workplace location -> Latitude: {lat_workplace}, Longitude: {lon_workplace}")

# Function to calculate straight-line (geodesic) distance
def calculate_distance(lat1, lon1, lat2, lon2):
    try:
        if pd.notnull(lat1) and pd.notnull(lon1):
            return geodesic((lat1, lon1), (lat2, lon2)).km
    except:
        return None
    return None

# Calculate distance from each employee to the workplace
df['distance_km'] = df.apply(
    lambda row: calculate_distance(row['latitude'], row['longitude'], lat_workplace, lon_workplace),
    axis=1
)

# Save final file with distances
df.to_csv(r"C:\Users\pheli\Downloads\employees_distance.csv", index=False)
print("✅ Final file saved as 'employees_distance.csv'")
